# Function that generate panel data from raw data

In [ ]:
import pandas as pd
import numpy as np
import recordlinkage
from recordlinkage.preprocessing import clean
import matplotlib.pyplot as plt
from fuzzywuzzy import fuzz
import seaborn

In [ ]:
df = pd.read_csv("input/raw_data_clean.csv", sep=";", encoding="utf-8")

In [ ]:
def create_panel_data(df):
    """
    Create panel data from a single DataFrame containing multiple years.
    Assigns consistent person IDs across years using name matching.
    
    Parameters:
    df: DataFrame with columns 'year', 'name', 'ministry', 'position', 'abbreviated_name', 
                              'is_abbreviated'
    
    Returns:
    DataFrame with added 'person_id' column
    """
    # Make a copy of the input data
    df_panel = df.copy()
    
    # Sort by year to ensure we process in chronological order
    df_panel = df_panel.sort_values('year')
    
    # Get unique years in ascending order
    years = sorted(df_panel['year'].unique())
    
    # Initialize person_id for the first year
    first_year_data = df_panel[df_panel['year'] == years[0]]
    first_year_indices = first_year_data.index
    
    # Assign initial person IDs (1 to n) for the first year
    df_panel.loc[first_year_indices, 'person_id'] = range(1, len(first_year_indices) + 1)
    
    # Process each subsequent year
    for i in range(1, len(years)):
        current_year = years[i]
        prev_year = years[i-1]
        
        print(f"\nProcessing year {current_year}...")
        
        # Get data for the current and previous years
        prev_year_data = df_panel[df_panel['year'] == prev_year].copy()
        current_year_data = df_panel[df_panel['year'] == current_year].copy()
        
        # Clean names
        prev_year_data['clean_name'] = clean(prev_year_data['name'])
        current_year_data['clean_name'] = clean(current_year_data['name'])
        
        # Reset indices for matching
        prev_year_data = prev_year_data.reset_index()
        current_year_data = current_year_data.reset_index()
        
        # Store original indices for updating the main DataFrame later
        orig_indices = {i: row['index'] for i, row in current_year_data.iterrows()}
        
        # ----- NAME SIMILARITY PREPROCESSING -----
        
        # Pre-compute all pairwise name similarities
        all_name_similarities = {}
        for curr_idx, curr_row in current_year_data.iterrows():
            for prev_idx, prev_row in prev_year_data.iterrows():
                # Calculate token sort ratio for better handling of name component ordering
                similarity = fuzz.token_sort_ratio(curr_row['name'], prev_row['name'])
                all_name_similarities[(curr_idx, prev_idx)] = similarity / 100.0  # Normalize to 0-1
                
                # If the similarity is too low (< 50%), we won't consider this a potential match
                if similarity < 50:
                    all_name_similarities[(curr_idx, prev_idx)] = 0.0
        
        # Initialize person IDs for current year
        current_year_person_ids = pd.Series([None] * len(current_year_data), index=current_year_data.index)
        
        # Track which person IDs from previous year have already been assigned
        used_prev_ids = set()
        
        # Track which current indices have been matched
        matched_current_indices = set()
        
        ### STEP 1: WITHIN MINISTRY NAME MATCHING ###
        print("Step 1: Within ministry name matching...")
        
        # Create an indexer for within-ministry matching
        indexer = recordlinkage.Index()
        indexer.block('ministry')
        candidate_links_within_ministry = indexer.index(current_year_data, prev_year_data)
        
        # Create a comparison object
        comparison_within = recordlinkage.Compare()
        
        # Define comparison methods for different fields
        comparison_within.string('abbreviated_name', 'abbreviated_name', method='jarowinkler', threshold=0.9)
        comparison_within.exact('ministry', 'ministry')
        
        # Compute features
        features_within = comparison_within.compute(candidate_links_within_ministry, current_year_data, prev_year_data)
        
        # Add custom name similarity
        name_similarities = []
        for idx_pair in candidate_links_within_ministry:
            name_similarities.append(all_name_similarities.get(tuple(idx_pair), 0.0))
        
        features_within['name_sim'] = name_similarities
        
        # Only continue with pairs that have some name similarity
        valid_pairs = features_within['name_sim'] > 0
        features_within = features_within[valid_pairs]
        candidate_links_within_ministry = candidate_links_within_ministry[valid_pairs]
        
        if len(features_within) > 0:
            # Add the candidate pairs to the features dataframe for easier reference
            features_within['current_idx'] = [pair[0] for pair in candidate_links_within_ministry]
            features_within['prev_idx'] = [pair[1] for pair in candidate_links_within_ministry]
            
            # Calculate total score - no position factor here
            features_within['total_score'] = features_within['name_sim'] * 2.0  # Double weight for name
            
            # Add bonuses for exact matches
            if 'ministry' in features_within.columns:
                features_within['total_score'] += features_within['ministry'] * 0.5
                
            if 'abbreviated_name' in features_within.columns:
                features_within['total_score'] += features_within['abbreviated_name'] * 0.5
            
            # Adjusted threshold since we removed position
            threshold = 1.5  # Lowered from 2.0
            
            # Sort by total score to ensure best matches are processed first
            features_within = features_within.sort_values('total_score', ascending=False)
            
            # Process pairs in order of match quality
            for idx, row in features_within.iterrows():
                if row['total_score'] <= threshold:
                    continue
                    
                current_idx = int(row['current_idx'])
                prev_idx = int(row['prev_idx'])
                
                # Skip if current record already matched or previous person already matched
                prev_id = prev_year_data.loc[prev_idx, 'person_id']
                if current_idx in matched_current_indices or prev_id in used_prev_ids:
                    continue
                
                # Final name similarity check
                name_sim = all_name_similarities.get((current_idx, prev_idx), 0)
                if name_sim < 0.5:
                    print(f"Rejecting match due to low name similarity ({name_sim}): {current_year_data.loc[current_idx, 'name']} vs {prev_year_data.loc[prev_idx, 'name']}")
                    continue
                
                # Assign person ID from previous year
                current_year_person_ids[current_idx] = prev_id
                
                # Mark as matched
                used_prev_ids.add(prev_id)
                matched_current_indices.add(current_idx)
        
        ### STEP 2: WITHIN MINISTRY ABBREVIATED_NAME TO NAME MATCHING ###
        print("Step 2: Within ministry abbreviated name to name matching...")
        
        # Identify unmatched records from current year that have is_abbreviated = True
        unmatched_indices = current_year_data.index[
            (current_year_person_ids.isna()) &
            (current_year_data['is_abbreviated'] == True)
        ]
        
        if len(unmatched_indices) > 0:
            # Find records from previous year where is_abbreviated = False
            prev_year_not_abbreviated = prev_year_data[prev_year_data['is_abbreviated'] == False]
            
            # Create pairs for potential matches (only within same ministry)
            matches_count = 0
            
            for curr_idx in unmatched_indices:
                curr_row = current_year_data.loc[curr_idx]
                curr_ministry = curr_row['ministry']
                curr_name = curr_row['name']
                
                # Find potential matches in prev year (same ministry, not abbreviated, not used)
                potential_matches = prev_year_not_abbreviated[
                    (prev_year_not_abbreviated['ministry'] == curr_ministry) &
                    (~prev_year_not_abbreviated['person_id'].isin(used_prev_ids))
                ]
                
                best_match_idx = None
                best_match_score = 0.0
                
                for prev_idx, prev_row in potential_matches.iterrows():
                    # Compare curr_name with prev_abbreviated_name
                    name_sim = fuzz.token_sort_ratio(curr_name, prev_row['abbreviated_name']) / 100.0
                    
                    if name_sim >= 0.6 and name_sim > best_match_score:
                        best_match_idx = prev_idx
                        best_match_score = name_sim
                
                if best_match_idx is not None:
                    prev_id = prev_year_data.loc[best_match_idx, 'person_id']
                    
                    print(f"Abbreviated match within ministry: {curr_row['name']} -> {prev_year_data.loc[best_match_idx, 'name']} ({prev_year_data.loc[best_match_idx, 'abbreviated_name']})")
                    
                    current_year_person_ids[curr_idx] = prev_id
                    used_prev_ids.add(prev_id)
                    matched_current_indices.add(curr_idx)
                    matches_count += 1
            
            print(f"Step 2: Matched {matches_count} records using abbreviated name within ministry.")
        
        ### STEP 3: CROSS-MINISTRY NAME MATCHING ###
        print("Step 3: Cross-ministry name matching...")
        
        # Identify remaining unmatched records
        unmatched_indices = current_year_data.index[~current_year_data.index.isin(matched_current_indices)]
        
        if len(unmatched_indices) > 0:
            unmatched_data = current_year_data.loc[unmatched_indices]
            
            # Create a new indexer for cross-ministry matching
            cross_indexer = recordlinkage.Index()
            cross_indexer.full()  # Compare all pairs
            cross_candidate_links = cross_indexer.index(unmatched_data, prev_year_data)
            
            # Create stricter comparison for cross-ministry
            cross_comparison = recordlinkage.Compare()
            cross_comparison.string('abbreviated_name', 'abbreviated_name', method='jarowinkler', threshold=0.95)
            
            # Compute features
            cross_features = cross_comparison.compute(cross_candidate_links, unmatched_data, prev_year_data)
            
            # Add custom name similarity
            cross_name_similarities = []
            for idx_pair in cross_candidate_links:
                cross_name_similarities.append(all_name_similarities.get(tuple(idx_pair), 0.0))
            
            cross_features['name_sim'] = cross_name_similarities
            
            # Even stricter name similarity for cross-ministry (minimum 70%)
            valid_pairs = cross_features['name_sim'] >= 0.7
            cross_features = cross_features[valid_pairs]
            cross_candidate_links = cross_candidate_links[valid_pairs]
            
            if len(cross_features) > 0:
                # Add candidate pairs to features
                cross_features['current_idx'] = [pair[0] for pair in cross_candidate_links]
                cross_features['prev_idx'] = [pair[1] for pair in cross_candidate_links]
                
                # Calculate total score - no position factor
                cross_features['total_score'] = cross_features['name_sim'] * 3.0  # Triple weight for name
                
                if 'abbreviated_name' in cross_features.columns:
                    cross_features['total_score'] += cross_features['abbreviated_name'] * 0.5
                
                # Adjusted threshold
                threshold = 2.5  # Lowered from 3.0
                
                # Sort by total score
                cross_features = cross_features.sort_values('total_score', ascending=False)
                
                # Process pairs in order of match quality
                matches_count = 0
                for idx, row in cross_features.iterrows():
                    if row['total_score'] <= threshold:
                        continue
                        
                    current_idx = int(row['current_idx'])
                    prev_idx = int(row['prev_idx'])
                    
                    # Skip if previous person already matched
                    prev_id = prev_year_data.loc[prev_idx, 'person_id']
                    if prev_id in used_prev_ids:
                        continue
                    
                    # Double-check name similarity
                    if all_name_similarities.get((current_idx, prev_idx), 0) < 0.7:
                        print(f"Rejecting cross-ministry match due to low name similarity: {unmatched_data.loc[current_idx, 'name']} vs {prev_year_data.loc[prev_idx, 'name']}")
                        continue
                    
                    # Assign person ID
                    current_year_person_ids[current_idx] = prev_id
                    
                    # Mark as matched
                    used_prev_ids.add(prev_id)
                    matched_current_indices.add(current_idx)
                    matches_count += 1
                
                print(f"Step 3: Matched {matches_count} records using cross-ministry name matching.")
        
        ### STEP 4: CROSS-MINISTRY ABBREVIATED_NAME TO NAME MATCHING ####
        print("Step 4: Cross-ministry abbreviated name to name matching...")
        
        # Identify remaining unmatched records that are abbreviated
        unmatched_indices = current_year_data.index[
            (~current_year_data.index.isin(matched_current_indices)) &
            (current_year_data['is_abbreviated'] == True)
        ]
        
        if len(unmatched_indices) > 0:
            # Find records from previous year that are not abbreviated and not used
            prev_year_not_abbreviated = prev_year_data[
                (prev_year_data['is_abbreviated'] == False) &
                (~prev_year_data['person_id'].isin(used_prev_ids))
            ]
            
            matches_count = 0
            
            for curr_idx in unmatched_indices:
                curr_row = current_year_data.loc[curr_idx]
                curr_name = curr_row['name']
                
                best_match_idx = None
                best_match_score = 0.0
                
                # Try to match with all non-abbreviated prev year records
                for prev_idx, prev_row in prev_year_not_abbreviated.iterrows():
                    # Compare curr_name with prev_abbreviated_name
                    name_sim = fuzz.token_sort_ratio(curr_name, prev_row['abbreviated_name']) / 100.0
                    
                    # Require higher threshold for cross-ministry
                    if name_sim >= 0.7 and name_sim > best_match_score:
                        best_match_idx = prev_idx
                        best_match_score = name_sim
                
                if best_match_idx is not None:
                    prev_id = prev_year_data.loc[best_match_idx, 'person_id']
                    
                    print(f"Abbreviated match across ministries: {curr_row['name']} ({curr_row['ministry']}) -> {prev_year_data.loc[best_match_idx, 'name']} ({prev_year_data.loc[best_match_idx, 'ministry']}, {prev_year_data.loc[best_match_idx, 'abbreviated_name']})")
                    
                    current_year_person_ids[curr_idx] = prev_id
                    used_prev_ids.add(prev_id)
                    matched_current_indices.add(curr_idx)
                    matches_count += 1
            
            print(f"Step 4: Matched {matches_count} records using abbreviated name across ministries.")
        
        ### ASSIGN NEW IDS FOR REMAINING UNMATCHED RECORDS ####
        
        # Identify final unmatched records
        unmatched_indices = current_year_data.index[~current_year_data.index.isin(matched_current_indices)]
        
        if len(unmatched_indices) > 0:
            # Get max person ID used so far
            max_id = df_panel['person_id'].max()
            
            # Assign new sequential IDs
            for i, idx in enumerate(unmatched_indices):
                current_year_person_ids[idx] = max_id + i + 1
        
        ### UPDATE MAIN DATAFRAME WITH PERSON IDS ###
        
        # Map back to original indices
        for curr_idx, person_id in current_year_person_ids.items():
            if pd.notna(person_id):
                orig_idx = orig_indices[curr_idx]
                df_panel.loc[orig_idx, 'person_id'] = int(person_id)
        
        # Print matching summary
        matched_count = len(current_year_data) - len(unmatched_indices)
        new_count = len(unmatched_indices)
        print(f"Year {current_year}: matched {matched_count} records with previous year. Added {new_count} new IDs.")
        
        # Print details of new entries
        if new_count > 0:
            print("New entries:")
            for idx in unmatched_indices:
                orig_idx = orig_indices[idx]
                row = current_year_data.loc[idx]
                print(f"  - {row['name']} ({row['ministry']}, {row['position']})")
    
    # Ensure person_id is integer
    df_panel['person_id'] = df_panel['person_id'].astype(int)
    
    # Verify the unique constraint - each person should appear at most once per year
    for year in years:
        year_data = df_panel[df_panel['year'] == year]
        id_counts = year_data['person_id'].value_counts()
        
        if any(id_counts > 1):
            duplicate_ids = id_counts[id_counts > 1].index.tolist()
            print(f"WARNING: Constraint violation - person IDs appearing multiple times in year {year}: {duplicate_ids}")
            
            # Fix any duplicates
            for dup_id in duplicate_ids:
                dup_indices = year_data[year_data['person_id'] == dup_id].index[1:]
                for i, idx in enumerate(dup_indices):
                    max_id = df_panel['person_id'].max()
                    df_panel.loc[idx, 'person_id'] = max_id + 1
                    print(f"Assigned new ID {max_id + 1} to duplicate record.")
    
    return df_panel

In [ ]:
panel = create_panel_data(df)